In [1]:
import pandas as pd  
from random import random
 
pdata = pd.read_csv('oct_march.csv',names=['chid','views','subcriber','videocount','date']) 


In [2]:
pdata=pdata.drop(pdata.index[0])

In [3]:
channel_groups=pdata.groupby('chid')
grps=[channel_groups.get_group(x) for x in channel_groups.groups]

In [4]:

def _load_data(data, n_prev = 4):  
    """
    data should be pd.DataFrame()
    """

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data.iloc[i:i+n_prev-1].as_matrix())
        docY.append(data.iloc[i+n_prev].as_matrix())
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

def train_test_split(df, test_size=0):  
    """
    This just splits data to training and testing parts
    """
    ntrn = round(len(df) * (1 - test_size))

    X_train, y_train = _load_data(df.iloc[0:ntrn])

    return (X_train, y_train)

In [5]:
def m():
    in_out_neurons = 3  

    hidden_neurons = 300


    model = Sequential()  

    model.add(LSTM(batch_input_shape=(1,3,3),units=hidden_neurons, return_sequences=True,stateful=True)) 
    model.add(Dropout(0.2))
    model.add(LSTM(units=150,stateful=True))
    model.add(Dropout(0.2))
    model.add(Dense(units=10))  
    model.add(Activation("tanh"))
    model.add(Dropout(0.2))
    model.add(Dense(units=3))
    model.add(Activation("linear"))
    model.add(Dropout(0.2))
    model.compile(loss="mean_squared_error", optimizer="rmsprop") 
    return model

In [6]:
%matplotlib notebook
from keras.models import Sequential  
from keras.layers.core import Dense, Activation,Dropout 
from keras.layers.recurrent import LSTM

import numpy as np
l=np.ndarray((len(grps),3,2))
for i in range(len(grps)):
    p=grps[i]
    p=p.sort_values(by='date')
    del p['date']
    del p['chid']
    p['views']=p.views.astype(int)
    l[i][0][0]=p['views'].max()- p['views'].min()
    l[i][0][1]=np.asarray(p['views'], dtype=np.float).mean()
    p['views']= (p['views'] - l[i][0][1] )/ l[i][0][0]
    p['subcriber']=p['subcriber'].astype(int)
    l[i][1][0]=p['subcriber'].max() - p['subcriber'].min()
    l[i][1][1]=np.asarray(p['subcriber'], dtype=np.float).mean()
    p['subcriber']= (p['subcriber'] - np.asarray(p['subcriber'], dtype=np.float).mean()) / (p['subcriber'].max() - p['subcriber'].min())
    p['videocount']=p['videocount'].astype(int)
    l[i][2][0]=p['videocount'].max() - p['videocount'].min()
    l[i][2][1]=np.asarray(p['videocount'], dtype=np.float).mean()
    p['videocount']= (p['videocount'] - np.asarray(p['videocount'], dtype=np.float).mean()) / (p['videocount'].max() - p['videocount'].min())
    (X_train, y_train)= train_test_split(p)  # retrieve data
    X_train=X_train.reshape((-1,3,3))
    M=m() 
    M.fit(x=X_train, y=y_train,batch_size=1,epochs=3, verbose=2,shuffle=False) 
    M.reset_states()

Using TensorFlow backend.


Epoch 1/3
3s - loss: nan
Epoch 2/3
2s - loss: nan
Epoch 3/3
2s - loss: nan
Epoch 1/3
3s - loss: nan
Epoch 2/3
2s - loss: nan
Epoch 3/3
2s - loss: nan
Epoch 1/3
3s - loss: 0.0344
Epoch 2/3
2s - loss: 0.0292
Epoch 3/3
2s - loss: 0.0304
Epoch 1/3
4s - loss: 0.0338
Epoch 2/3
2s - loss: 0.0272
Epoch 3/3
2s - loss: 0.0258
Epoch 1/3
4s - loss: 0.0262
Epoch 2/3
3s - loss: 0.0260
Epoch 3/3
4s - loss: 0.0277
Epoch 1/3
4s - loss: 0.0603
Epoch 2/3
2s - loss: 0.0522
Epoch 3/3
2s - loss: 0.0519
Epoch 1/3
4s - loss: 0.0497
Epoch 2/3
3s - loss: 0.0630
Epoch 3/3
3s - loss: 0.0558
Epoch 1/3
4s - loss: 0.0410
Epoch 2/3
2s - loss: 0.0363
Epoch 3/3
2s - loss: 0.0414
Epoch 1/3
4s - loss: 0.0442
Epoch 2/3
2s - loss: 0.0376
Epoch 3/3
2s - loss: 0.0363
Epoch 1/3
4s - loss: 0.0415
Epoch 2/3
2s - loss: 0.0433
Epoch 3/3
3s - loss: 0.0305
Epoch 1/3
4s - loss: 0.0250
Epoch 2/3
2s - loss: 0.0215
Epoch 3/3
2s - loss: 0.0232
Epoch 1/3
5s - loss: nan
Epoch 2/3
4s - loss: nan
Epoch 3/3
3s - loss: nan
Epoch 1/3
4s - loss

In [7]:
import math
from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2017, 4, 1)
end_date = date(2017, 7, 1)
p=np.empty((10*31,3,3))
predicted=np.ndarray((10*31,3))
validation=np.empty((10*31,3))
g=grps[0]
g['views']=g.views.astype(int)
g['subcriber']=g.subcriber.astype(int)
g['videocount']=g.videocount.astype(int)
X=g.sort_values(by='date')
print ("prediction for "+X['chid'].values[0])
del g['date']
del g['chid']
del X['date']
del X['chid']
M.reset_states()
X=X.head(3)
p[0]=X.values.reshape((1,3,3))
i=1
for k in range(3):
    validation[k]=(X.values)[k]
for single_date in daterange(date(2016,10,1), date(2017,3,31)):
    p[i][0]=p[i-1][1]
    p[i][1]=p[i-1][2]
    p[i][2]=M.predict(p[i-1].reshape((1,3,3)))
    validation[i+3]=p[i][2]
    i=i+1
print (i)
print (X.values.shape)
print (validation.shape)


/home/shubham/anaconda3/envs/tensorflowcpu/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/shubham/anaconda3/envs/tensorflowcpu/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/shubham/anaconda3/envs/tensorflowcpu/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

prediction for UC60GHVk_6ddgqhHwcg
182
(3, 3)
(310, 3)


In [8]:
for x in range(i):
    sqr_err=(g.values)[x]-validation[x]
    mse=np.mean(sqr_err)
    rmse=math.sqrt(mse)
print (rmse)
j=0
for single_date in daterange(start_date, end_date):
    p[i][0]=p[i-1][1]
    p[i][1]=p[i-1][2]
    p[i][2]=M.predict(p[i-1].reshape((1,3,3)))
    predicted[j][0]=p[i][2][0]*l[0][0][0]+l[0][0][1]
    predicted[j][1]=p[i][2][1]*l[0][1][0]+l[0][1][1]
    predicted[j][2]=p[i][2][2]*l[0][2][0]+l[0][2][1]
    print (single_date,predicted[i])
    i=i+1

1177.5891366405253
2017-04-01 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-02 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-03 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-04 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-05 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-06 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-07 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-08 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-09 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-10 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-11 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-12 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-13 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-14 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-15 [  3.88548855e+06   2.79812409e+05   7.00000000e+00]
2017-04-16 [  3.88548

In [9]:
l

array([[[  2.05910000e+04,   3.87713807e+06],
        [  7.00800000e+03,   2.77014758e+05],
        [  0.00000000e+00,   7.00000000e+00]],

       [[  5.04750000e+04,   9.39927473e+04],
        [  2.85000000e+02,   2.35290659e+03],
        [  0.00000000e+00,   1.30000000e+01]],

       [[  4.72935000e+05,   2.28548357e+06],
        [  1.77100000e+03,   1.58348132e+04],
        [  9.80000000e+01,   6.37835165e+02]],

       [[  2.56954000e+05,   1.57314719e+06],
        [  6.20000000e+01,   1.13318681e+03],
        [  1.60000000e+01,   3.73620879e+02]],

       [[  1.02979000e+05,   2.41532143e+06],
        [  3.99000000e+02,   3.76683516e+03],
        [  2.60000000e+01,   3.01626374e+02]],

       [[  9.64830000e+04,   3.32746549e+05],
        [  3.17000000e+02,   2.48075275e+03],
        [  1.50000000e+01,   1.07950549e+02]],

       [[  4.80858000e+05,   1.55706314e+06],
        [  1.60000000e+02,   1.39624176e+03],
        [  1.00000000e+00,   2.22351648e+02]],

       [[  1.8857400

In [11]:
predicted[1][0]

1.4171933291892825e-314

In [28]:
print ("{:.0f}".format(float(predicted[i-1][0])))

3885489


In [27]:
grps[0]

,views,subcriber,videocount
11570,3866163,280396,7
11571,3866266,280349,7
11572,3866443,280316,7
11573,3866623,280288,7
11574,3866736,280265,7
11575,3866871,280222,7
11576,3866983,280194,7
11577,3867101,280182,7
11578,3867240,280124,7
11579,3867384,280094,7
